<a href="https://colab.research.google.com/github/ancxz/ancxz/blob/main/OceanPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#1 conectanto no sqlite no colab

In [2]:
import sqlite3
con = sqlite3.connect('loja.db')
cursor = con.cursor ()
cursor.execute("DROP TABLE IF EXISTS produto")
cursor.execute("DROP TABLE IF EXISTS pedido")


In [3]:
#2 Criando as tabelas

In [4]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS pedido(
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  cliente TEXT NOT NULL,
  data_pedido DATE NOT NULL
);
""")
con.commit()

In [5]:
cursor.execute("""
CREATE TABLE produto(
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  nome TEXT NOT NULL,
  preco REAL NOT NULL,
  pedido_id INTEGER NOT NULL,
  FOREIGN KEY(pedido_id) REFERENCES pedido(id)
);
""")
con.commit()

In [6]:
#3 Alter Table

In [7]:
cursor.execute("""
ALTER TABLE pedido ADD COLUMN forma_pagamento TEXT;
""")
con.commit()

In [8]:
# Insert

In [9]:
cursor.execute("DROP TABLE IF EXISTS pedido;")
cursor.execute("""
CREATE TABLE IF NOT EXISTS pedido(
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  cliente TEXT NOT NULL,
  data_pedido DATE NOT NULL
);
""")
try:
    cursor.execute("""
    ALTER TABLE pedido ADD COLUMN forma_pagamento TEXT;
    """)
except sqlite3.OperationalError as e:
    if "duplicate column name" not in str(e):
        raise e
con.commit()

cursor.execute("""
INSERT INTO pedido (cliente, data_pedido, forma_pagamento)
VALUES ('Vitor Felipe', '2025-11-28', 'PIX')""")
con.commit()

In [10]:
pedidos = [
    ("Luiz Felipe", '2025-10-15', 'Em processamento', 'Cartão de Crédito'),
    ('Gloria', '2025-01-15', 'Criado', 'Boleto')
]

try:
    cursor.execute("""
    ALTER TABLE pedido ADD COLUMN status TEXT;
    """)
except sqlite3.OperationalError as e:
    if "duplicate column name" not in str(e):
        raise e
con.commit()

cursor.executemany("""
INSERT INTO pedido (cliente, data_pedido, status, forma_pagamento)
VALUES (?,?,?,?)""", pedidos)
con.commit()

In [11]:
produtos = [
    ("Notebook", 4500.00, 1),
    ("Mouse", 150.00, 1),
    ("Teclado", 250.00, 2),
    ("Monitor", 750.00, 3),
]

cursor.executemany("""
INSERT INTO produto (nome, preco, pedido_id)
VALUES (?,?,?)""", produtos)
con.commit()

In [12]:
cursor.execute("""
UPDATE pedido SET status = 'Finalizado' WHERE id = 2
""")
con.commit()

In [13]:
cursor.execute("""
DELETE FROM produto WHERE nome = 'Mouse'
""")
con.commit()

In [14]:
cursor.execute("""
SELECT * FROM pedido
""")
for linha in cursor.fetchall():
    print(linha)

(1, 'Vitor Felipe', '2025-11-28', 'PIX', None)
(2, 'Luiz Felipe', '2025-10-15', 'Cartão de Crédito', 'Finalizado')
(3, 'Gloria', '2025-01-15', 'Boleto', 'Criado')


In [15]:
cursor.execute("SELECT * FROM pedido")
print(cursor.fetchall())

[(1, 'Vitor Felipe', '2025-11-28', 'PIX', None), (2, 'Luiz Felipe', '2025-10-15', 'Cartão de Crédito', 'Finalizado'), (3, 'Gloria', '2025-01-15', 'Boleto', 'Criado')]


In [16]:
cursor.execute("SELECT * FROM pedido")
# print(cursor.fetchall())
# print(cursor.fetchmany(1))
linha = cursor.fetchone()

# Get column names from cursor description
column_names = [description[0] for description in cursor.description]
linha_dict = dict(zip(column_names, linha))

print("Linha completa", linha_dict)
print("Cliente", linha_dict["cliente"])
print("Data do Pedido", linha_dict["data_pedido"])
print("Status", linha_dict["status"])
print("Pagamento", linha_dict["forma_pagamento"])

Linha completa {'id': 1, 'cliente': 'Vitor Felipe', 'data_pedido': '2025-11-28', 'forma_pagamento': 'PIX', 'status': None}
Cliente Vitor Felipe
Data do Pedido 2025-11-28
Status None
Pagamento PIX


In [17]:
cursor.execute("""
SELECT
pedido.cliente,
pedido.data_pedido,
pedido.status,
produto.nome,
produto.preco
FROM pedido
INNER JOIN produto ON pedido.id = produto.pedido_id
""")
for linha in cursor.fetchall():
    print(linha)

('Vitor Felipe', '2025-11-28', None, 'Notebook', 4500.0)
('Luiz Felipe', '2025-10-15', 'Finalizado', 'Teclado', 250.0)
('Gloria', '2025-01-15', 'Criado', 'Monitor', 750.0)


In [18]:
cursor.execute("""
SELECT * FROM   pedido WHERE status = 'Finalizado'
""")
for linha in cursor.fetchall():
    print(linha)

(2, 'Luiz Felipe', '2025-10-15', 'Cartão de Crédito', 'Finalizado')


In [19]:
#SQLAlchemy -> RAW

In [20]:
from sqlalchemy import create_engine, text
engine = create_engine("sqlite:///zoo.db")
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS animais"))

In [21]:
with engine.begin() as conn:
    conn.execute (text("""
     CREATE TABLE animais(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT,
    especie TEXT,
    idade INTEGER)""") )

In [22]:
with engine.begin() as conn:
  conn.execute(text("""
  INSERT INTO animais (nome, especie, idade)
  VALUES ('Rex', 'Cachorro', 3)"""))

In [23]:
with engine.begin() as conn:
  rs = conn.execute(text("SELECT * FROM animais"))
  for r in rs:
    print(r)

(1, 'Rex', 'Cachorro', 3)


In [24]:
with engine.begin() as conn:
  conn.execute(text("""
INSERT INTO animais (nome, especie, idade)
VALUES
('Luna', 'Gato', 2),
('Max', 'Cachorro', 4),
('Bella', 'Gato', 1),
('Simba', 'Cachorro', 3),
('Oliver', 'Cachorro', 2),
('Milo', 'Cachorro', 1),
('Nala', 'Cachorro', 2)

"""))

In [25]:
with engine.begin() as conn:
  rs = conn.execute(text("SELECT * FROM animais"))
  for r in rs:
    print(r)

(1, 'Rex', 'Cachorro', 3)
(2, 'Luna', 'Gato', 2)
(3, 'Max', 'Cachorro', 4)
(4, 'Bella', 'Gato', 1)
(5, 'Simba', 'Cachorro', 3)
(6, 'Oliver', 'Cachorro', 2)
(7, 'Milo', 'Cachorro', 1)
(8, 'Nala', 'Cachorro', 2)


In [26]:
with engine.begin() as conn:
  rs = conn.execute(text("SELECT * FROM animais"))
  for r in rs:
    print(r)

(1, 'Rex', 'Cachorro', 3)
(2, 'Luna', 'Gato', 2)
(3, 'Max', 'Cachorro', 4)
(4, 'Bella', 'Gato', 1)
(5, 'Simba', 'Cachorro', 3)
(6, 'Oliver', 'Cachorro', 2)
(7, 'Milo', 'Cachorro', 1)
(8, 'Nala', 'Cachorro', 2)


In [27]:
#SQL Expression Language

In [28]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, select
engine = create_engine("sqlite:///brasil.db")
MetaData = MetaData()

In [29]:
metadata_obj = MetaData()
estados = Table(
    "estados", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("nome", String),
    Column("sigla", String)
)

TypeError: 'MetaData' object is not callable

In [ ]:
MetaData.create_all (engine)

In [ ]:
from sqlalchemy import insert, create_engine, MetaData, Table, Column, Integer, String

# Ensure engine is available and points to the correct database
# (Assuming 'engine' is global, but explicitly defining for self-containment)
engine = create_engine("sqlite:///brasil.db")

# Create a local MetaData object
metadata_obj = MetaData()

# Define the 'estados' table with this local metadata_obj
estados = Table(
    "estados", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("nome", String),
    Column("sigla", String)
)

# Create the table(s) associated with this metadata_obj, if they don't already exist
metadata_obj.create_all(engine, checkfirst=True)

with engine.begin() as conn:
  conn.execute(insert(estados).values(nome="São Paulo", sigla="SP"))

In [ ]:
with engine.connect() as con:
  con.execute(insert(estados).values(nome="Pernambuco", sigla = "PE"))
  rs = con.execute(select(estados))
  print(rs.fetchone())

In [ ]:
#ORM – Pedidos e Produtos (1 Pedido → N Produtos)

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

# Base ORM
Base = declarative_base()

In [ ]:
# Classe Pedido
class Pedido(Base):
    __tablename__ = "pedidos"

    id = Column(Integer, primary_key=True)
    cliente = Column(String)

    # Relacionamento com produtos
    produtos = relationship("Produto", back_populates="pedido")

In [ ]:
# Classe Produto
class Produto(Base):
    __tablename__ = "produtos"

    id = Column(Integer, primary_key=True)
    nome = Column(String)
    preco = Column(Integer)
    pedido_id = Column(Integer, ForeignKey("pedidos.id"))

    # Relacionamento com pedidos
    pedido = relationship("Pedido", back_populates="produtos")

In [ ]:
# Cria banco SQLite
engine = create_engine("sqlite:///vendas.db")

In [ ]:
# Cria as tabelas
Base.metadata.create_all(engine)

In [ ]:
# Cria a sessão
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# ===============================
# INSERT
# ===============================

# Cria pedidos
pedido1 = Pedido(cliente="Danilo")
pedido2 = Pedido(cliente="Maria")

# Cria produtos
produto1 = Produto(nome="Teclado", preco=150, pedido=pedido1)
produto2 = Produto(nome="Mouse", preco=80, pedido=pedido1)
produto3 = Produto(nome="Monitor", preco=1200, pedido=pedido2)

# Adiciona tudo à sessão
session.add_all([pedido1, pedido2, produto1, produto2, produto3])
session.commit()

In [ ]:
# Busca pedidos com seus produtos
pedidos = session.query(Pedido).all()

for pedido in pedidos:
    print(f"\nPedido {pedido.id} - Cliente: {pedido.cliente}")
    for produto in pedido.produtos:
        print(f"  Produto: {produto.nome} - R$ {produto.preco}")